In [2]:
import numpy as np
from itertools import product
from tqdm import tqdm  # For progress bar
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def generate_all_allocations(num_agents, num_items):
    """
    Generate all possible allocations of items to agents as a 3D numpy array.
    
    Args:
        num_agents (int): Number of agents
        num_items (int): Number of items
        
    Returns:
        numpy.ndarray: 3D array of shape (num_allocations, num_agents, num_items)
                      where a 1 indicates an item is assigned to an agent
    """
    # Calculate the total number of possible allocations
    num_allocations = num_agents ** num_items
    
    # Create an empty 3D numpy array to store all allocations
    allocations = np.zeros((num_allocations, num_agents, num_items), dtype=int)
    
    # Generate all possible assignments of items to agents
    all_assignments = list(product(range(num_agents), repeat=num_items))
    
    # Fill the allocations array
    for alloc_idx, assignment in enumerate(all_assignments):
        for item, agent in enumerate(assignment):
            allocations[alloc_idx, agent, item] = 1
            
    return allocations

def calculate_utilities(allocations, valuations):
    """
    Calculate the utility for each agent for each allocation.
    
    Args:
        allocations (numpy.ndarray): 3D array of shape (num_allocations, num_agents, num_items)
        valuations (numpy.ndarray): 2D array of shape (num_agents, num_items)
        
    Returns:
        numpy.ndarray: 2D array of shape (num_allocations, num_agents) with utility values
    """
    num_allocations, num_agents, num_items = allocations.shape
    
    # Initialize utilities array
    utilities = np.zeros((num_allocations, num_agents))
    
    # For each allocation, calculate utility for each agent
    for alloc_idx in range(num_allocations):
        for agent in range(num_agents):
            # Element-wise multiplication of allocation and valuations, then sum
            utilities[alloc_idx, agent] = np.sum(allocations[alloc_idx, agent] * valuations[agent])
    
    return utilities

def find_envy_free_allocations(allocations, utilities):
    """
    Find allocations that are envy-free (no agent prefers another agent's allocation).
    
    Args:
        allocations (numpy.ndarray): 3D array of allocations
        utilities (numpy.ndarray): 2D array of utilities
        
    Returns:
        numpy.ndarray: Boolean array indicating which allocations are envy-free
        numpy.ndarray: Filtered utilities for envy-free allocations
    """
    num_allocations, num_agents = utilities.shape
    is_envy_free = np.ones(num_allocations, dtype=bool)
    
    for alloc_idx in range(num_allocations):
        for agent in range(num_agents):
            # Get agent's utility for their allocation
            own_utility = utilities[alloc_idx, agent]
            
            # Check what utility the agent would get from others' allocations
            for other_agent in range(num_agents):
                if agent == other_agent:
                    continue
                
                # Calculate utility agent would get from other agent's allocation
                other_alloc = allocations[alloc_idx, other_agent]
                other_utility = 0
                for item in range(allocations.shape[2]):
                    if other_alloc[item] == 1:
                        other_utility += valuations[agent, item]
                
                # If agent envies another agent, this allocation is not envy-free
                if other_utility > own_utility:
                    is_envy_free[alloc_idx] = False
                    break
            
            if not is_envy_free[alloc_idx]:
                break
    
    # Filter utilities to only include envy-free allocations
    envy_free_utilities = utilities[is_envy_free]
    
    return is_envy_free, envy_free_utilities

def find_pareto_frontier(utilities):
    """
    Find the Pareto frontier from a set of utility vectors.
    
    Args:
        utilities (numpy.ndarray): 2D array of utilities
        
    Returns:
        numpy.ndarray: Boolean array indicating which allocations are Pareto optimal
        numpy.ndarray: Filtered utilities for Pareto optimal allocations
    """
    num_allocations = utilities.shape[0]
    is_pareto_optimal = np.ones(num_allocations, dtype=bool)
    
    for i in range(num_allocations):
        if not is_pareto_optimal[i]:
            continue
            
        for j in range(num_allocations):
            if i == j or not is_pareto_optimal[j]:
                continue
                
            # Check if j dominates i (all utilities in j >= i and at least one >)
            if np.all(utilities[j] >= utilities[i]) and np.any(utilities[j] > utilities[i]):
                is_pareto_optimal[i] = False
                break
    
    # Filter utilities to only include Pareto optimal allocations
    pareto_optimal_utilities = utilities[is_pareto_optimal]
    
    return is_pareto_optimal, pareto_optimal_utilities

def plot_pareto_frontier(utilities, title="Pareto Frontier"):
    """
    Plot the Pareto frontier for 2 or 3 agents.
    
    Args:
        utilities (numpy.ndarray): 2D array of Pareto optimal utilities
        title (str): Plot title
    """
    num_agents = utilities.shape[1]
    
    if num_agents == 2:
        plt.figure(figsize=(10, 6))
        plt.scatter(utilities[:, 0], utilities[:, 1], c='blue', marker='o')
        plt.xlabel('Utility for Agent 0')
        plt.ylabel('Utility for Agent 1')
        plt.title(title)
        plt.grid(True)
        plt.savefig('pareto_frontier_2d.png')
        plt.show()
        
    elif num_agents == 3:
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(utilities[:, 0], utilities[:, 1], utilities[:, 2], c='blue', marker='o')
        ax.set_xlabel('Utility for Agent 0')
        ax.set_ylabel('Utility for Agent 1')
        ax.set_zlabel('Utility for Agent 2')
        ax.set_title(title)
        plt.savefig('pareto_frontier_3d.png')
        plt.show()
        
    else:
        print(f"Plotting is only supported for 2 or 3 agents. You have {num_agents} agents.")
        print("First few Pareto optimal utility vectors:")
        for i in range(min(5, len(utilities))):
            print(f"Allocation {i}: {utilities[i]}")

# Example valuation table from the original problem

path = "outputs/agents_2/items_7/gpt4o/zero_shot0"
agents = 2
items = 7

envy_free_allocations = []

for i in range(100):

    with open(f"{path}/output_{i+1}.txt", "r") as file:
        # get allocation matrix

        # get valuation table find where 'Valuation Table:' and read the next lines
        try:
            lines = file.readlines()
            start = lines.index('Valuation Table:\n')
            end = lines.index('Output:\n')
            valuation_table = lines[start+1:end]
            valuation_table = ''.join(valuation_table)
            valuation_table = np.fromstring(valuation_table.replace('[', '').replace(']', ''), sep=' ')
            valuation_table = valuation_table.reshape(agents, items)
            valuation_table = valuation_table.astype(int)
        except:
            print(f"Error reading file {i+1}")
            continue



    valuations = valuation_table

    num_agents, num_items = valuations.shape

    # print(f"Generating all possible allocations for {num_agents} agents and {num_items} items...")
    # print(f"This will create {num_agents ** num_items} different allocations.")

    # Generate all possible allocations
    allocations = generate_all_allocations(num_agents, num_items)
    # print(f"Generated {allocations.shape[0]} allocations.")

    # Calculate utilities for all allocations
    # print("Calculating utilities for all allocations...")
    utilities = calculate_utilities(allocations, valuations)

    # Find envy-free allocations
    # print("Finding envy-free allocations...")
    is_envy_free, envy_free_utilities = find_envy_free_allocations(allocations, utilities)
    num_envy_free = np.sum(is_envy_free)
    print(f"Found {num_envy_free} envy-free allocations out of {allocations.shape[0]} total allocations.")

    envy_free_allocations.append(num_envy_free)

    # if num_envy_free == 0:
    #     print("No envy-free allocations found. Consider relaxing constraints or using a different approach.")

    # # Find Pareto optimal allocations among envy-free ones
    # print("Finding Pareto optimal allocations among envy-free allocations...")
    # is_pareto_optimal, pareto_optimal_utilities = find_pareto_frontier(envy_free_utilities)
    # num_pareto = np.sum(is_pareto_optimal)
    # print(f"Found {num_pareto} Pareto optimal allocations out of {num_envy_free} envy-free allocations.")

    # # Get the indices of the Pareto optimal allocations in the original allocation array
    # envy_free_indices = np.where(is_envy_free)[0]
    # pareto_indices = envy_free_indices[is_pareto_optimal]

    # # Print some sample Pareto optimal allocations
    # print("\nSample Pareto optimal allocations:")
    # for i in range(min(5, len(pareto_indices))):
    #     idx = pareto_indices[i]
    #     print(f"\nAllocation {i+1}:")
    #     for agent in range(num_agents):
    #         items = np.where(allocations[idx, agent] == 1)[0]
    #         print(f"  Agent {agent}: {list(items)}, Utility: {utilities[idx, agent]}")

    # # Plot the Pareto frontier if there are 2 or 3 agents
    # if num_agents <= 3:
    #     print("\nPlotting Pareto frontier...")
    #     plot_pareto_frontier(pareto_optimal_utilities, title="Pareto Frontier of Envy-Free Allocations")

    # break

    # Save results if desired
    # save_to_file = input("\nDo you want to save the Pareto optimal allocations to file? (y/n): ")
    # if save_to_file.lower() == 'y':
    #     np.save('pareto_optimal_allocations.npy', allocations[pareto_indices])
    #     np.save('pareto_optimal_utilities.npy', pareto_optimal_utilities)
    #     print("Saved Pareto optimal allocations and utilities to file.")


Found 15 envy-free allocations out of 128 total allocations.
Found 12 envy-free allocations out of 128 total allocations.
Found 8 envy-free allocations out of 128 total allocations.
Found 13 envy-free allocations out of 128 total allocations.
Found 20 envy-free allocations out of 128 total allocations.
Found 16 envy-free allocations out of 128 total allocations.
Found 23 envy-free allocations out of 128 total allocations.
Found 15 envy-free allocations out of 128 total allocations.
Found 9 envy-free allocations out of 128 total allocations.
Found 7 envy-free allocations out of 128 total allocations.
Found 6 envy-free allocations out of 128 total allocations.
Found 18 envy-free allocations out of 128 total allocations.
Found 18 envy-free allocations out of 128 total allocations.
Found 15 envy-free allocations out of 128 total allocations.
Found 16 envy-free allocations out of 128 total allocations.
Found 15 envy-free allocations out of 128 total allocations.
Found 14 envy-free allocatio

In [3]:
envy_free_allocations

[15,
 12,
 8,
 13,
 20,
 16,
 23,
 15,
 9,
 7,
 6,
 18,
 18,
 15,
 16,
 15,
 14,
 12,
 10,
 9,
 12,
 18,
 6,
 7,
 21,
 18,
 21,
 12,
 11,
 12,
 13,
 13,
 14,
 21,
 13,
 16,
 9,
 16,
 17,
 25,
 16,
 20,
 24,
 19,
 6,
 13,
 17,
 16,
 19,
 22,
 12,
 20,
 16,
 12,
 16,
 14,
 17,
 11,
 20,
 17,
 8,
 17,
 14,
 14,
 20,
 11,
 20,
 10,
 15,
 15,
 19,
 23,
 9,
 11,
 9,
 14,
 17,
 15,
 15,
 14,
 17,
 19,
 18,
 15,
 18,
 17,
 16,
 13,
 13,
 23,
 16,
 10,
 13,
 16,
 20,
 17,
 11,
 17,
 12,
 10]